In [1]:
using Revise
includet("../codebase/mlp.jl")

using Knet, Plots, Statistics, LinearAlgebra, Random, .mlp

In [ ]:
n=12 # length of input bit array
xdim = n
h1=h2=h3=2*n # hidden layer sizes
ydim = 1 # output is the probability for even parity
nbatch = 100 # number of bit strings presented at each iteration
ntest = div(2^n,8) # keep 1/8-th as test set
niter = 5 * 10^4 # number of training iterations

In [ ]:
# NN parameters
w1 = Param(randn(h1,xdim))
b1 = Param(randn(h1))
w2 = Param(randn(h2,h1))
b2 = Param(randn(h2))
w3 = Param(randn(h3,h2))
b3 = Param(randn(h3))
w4 = Param(randn(1,h3))
b4 = Param(randn(1))
w = (w1,w2,w3,w4)
b = (b1,b2,b3,b4)

In [ ]:
trainidx, testidx = mlp.gen_data(n, ntest)
xtrn, ytrn, xtst, ytst = mlp.split_data(trainidx, testidx, n, ntest)

In [ ]:
@time w, b, losstrn, losstst, rho = mlp.backprop(w, b, niter, trainidx, testidx, n, ntest, nbatch, (xdim, h1, h2, h3, ydim), rewire = true, LR = 0.1, p = 0.1);

In [ ]:
plot(losstrn)
plot!(losstst) 

In [ ]:
np=min(ntest,64)
scatter(pred(w,b,xtst)[1:np]
#    ,xlim=(0,np)
)
plot!(ytst[1:np])

In [ ]:
# accuracy
function acc(w,b,x,y)
    cnt = 0
    tot = length(x[1,:])
    for i=1:tot-1
        chk = (pred(w,b,x[:,i])[1] - y[i][1])^2
        if (chk < 0.5)
            cnt += 1
        end
    end
    return 1.0*cnt/tot
end

In [ ]:
acc(w,b,xtrn,ytrn),acc(w,b,xtst,ytst)

In [ ]:
sumtrn = 0.0; sumtst = 0.0

for i=1:100
    trainidx, testidx = mlp.gen_data(n, ntest)
    xtrn, ytrn, xtst, ytst = mlp.split_data(trainidx, testidx, n, ntest)
    sumtrn += acc(w, b, xtrn,ytrn); sumtst += acc(w, b, xtst,ytst)
end

(sumtrn/100,sumtst/100) # (overfitted but works)

In [ ]:
# distribution of weights
histogram(reshape(w1,:,1)[:,1],normalize=:pdf)

In [ ]:
heatmap(abs.(value(w2)),aspect_ratio=1)

In [ ]:
heatmap(abs.(value(w3)),aspect_ratio=1)

In [ ]:
heatmap(abs.(value(w4)),aspect_ratio=1)

In [ ]:
plot(rho)